In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import time


In [54]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

start_date = "2024.02.01"
end_date = "2024.04.30"
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start="  + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:니켈 전기차 배터리

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):50

크롤링할 종료 페이지:  50 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=1', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=11', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=21', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=31', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=41', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=51', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=61', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&

100%|███████████████████████████████████████████| 56/56 [00:15<00:00,  3.56it/s]

검색된 기사 갯수: 총  500 개

[뉴스 제목]
['"10 Minutes" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑', '투자 더 늘린다는 삼성SDI "전기차 장기 전망 밝아"', "K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷", '중국산 전기차 사고로 폭발, 문 열리지 않아 탑승자 전원 사망', "'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”", '엑셀 밟는 비야디 전기차…"연내 승용차 브랜드 론칭"', "'3천만원대' 기아 EV3 6월 출시…전기차 '캐즘' 돌파 선봉장", '"배터리 가격 경쟁력 갖추자"…이석희 SK온 사장 \'승부수\'', 'K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐', '현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다', 'K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진', "현대차·기아, CFO '전기차 동상이몽'...캐스퍼·EV3 전략 온도차", '“폴스타 전기차에 SK온 배터리 탑재·현지 생산…한국 기업과 협력 확대”', "'무임승차'에 뿔난 LG엔솔 '배터리 특허 전쟁' 선언…中 겨냥했나", '車업계, 전기차 신차 잇단 출시…캐즘 정면돌파 의지', 'LG엔솔, 전기차 부진에 결국 투자 규모 줄인다', "'英스포츠카' 로터스 품은 中 지리…'럭셔리 전기차'도 노린다", '"잠깐 쉬었다 멀리 가자"…K-배터리, 수요 살피며 속도조절', '포스코 “이차전지 소재 투자 일부 순연… 전기차 ‘캐즘’ 대응”', "SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상", '[인터뷰] 벤츠 최고기술책임자 "전동화전략 여전…전기차 할인경쟁 안해"', "포스코홀딩스, 1분기 영업익 17.3% 감소…배터리 투자 '브레이크'", "'캐즘'에 놀란 장인화號, 배터리 투자 늦추고 부진 사업 구조조정", '[배터리완전정복](33)中 LFP 대적할 3대 키워드…단결정·고전압·미드니켈', '[단독] 현대차 ‘아이오닉9’에 SK온 배터리 탑재된다', "전략특허 58% 뚫렸다…뿔난 LG

In [2]:
df1 = pd.read_csv('니켈 배터리_20240626_02시00분37초.csv')#니켈 배터리

In [3]:
df2 = pd.read_csv('니켈 원자재_20240626_02시01분47초.csv') #니켈 원자재

In [4]:
df3 = pd.read_csv('니켈 전기차 배터리_20240626_02시02분31초.csv') #니켈 전기차 배터리

In [5]:
import re

def clean_text(text):
    # HTML 태그 제거
    text = re.sub('<[^<]+?>', '', text)
    # 줄바꿈, 탭 문자 제거
    text = re.sub('[\r\n\t]', ' ', text)
    # 특수문자 제거 (예: ', ", -, ...)
    text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)
    # 중복 공백 제거
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [6]:
df1['Content'] = df1['content'].apply(clean_text)
df1 = df1[['date','title','Content']]
df2['Content'] = df2['content'].apply(clean_text)
df2 = df2[['date','title','Content']]
df3['Content'] = df3['content'].apply(clean_text)
df3 = df3[['date','title','Content']]


In [7]:
df1['Keyword'] = '니켈 배터리'
df2['Keyword'] = '니켈 원자재'
df3['Keyword'] = '니켈 전기차 배터리'


In [72]:
# 데이터프레임을 리스트로 묶어서 concat 함수에 전달하여 합침
df = pd.concat([df1, df2, df3], ignore_index=True)
df

,date,title,Content,Keyword
0,2024-04-30 14:26:01,"""10 Minutes"" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑",주행 가능한 폴스타 5 프로토타입에 시연10분 만에 중형 전기차 주행거리 320까지...,니켈 배터리
1,2024-04-30 13:14:19,"K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷",고객사 보수적 재고 운영에 메탈가 하락수익성에 악영향하반기 신차 출시 AMPC 확대...,니켈 배터리
2,2024-04-29 06:20:00,"""배터리 가격 경쟁력 갖추자""…이석희 SK온 사장 '승부수'",글로벌 전기차용 배터리 사용량그래픽임종철배터리 가격 경쟁력 확보에 주력해야 한다28...,니켈 배터리
3,2024-04-30 16:46:03,K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐,1Q 성적표 부진 속 전기차 캐즘 속 활로 모색LG에너지솔루션 삼성SDI SK온 등...,니켈 배터리
4,2024-04-30 11:30:01,현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다,현대차SK온 신차에 공급 협의SK온 서산공장 증설도 본격 돌입 SK온 서산 배터리공...,니켈 배터리
...,...,...,...,...
137,2024-04-04 10:45:04,6000만원 넘던 '모델3' 가격 확 내렸다…테슬라 '초강수',테슬라 신형 모델3 국내 상륙가격 5199만원부터 2017년 출시된 모델3 첫 페이...,니켈 전기차 배터리
138,2024-04-11 10:02:01,"K-배터리, 非중국 점유율 45.5%…SK온만 역성장",12월 국내 3사 합산 점유율 전년 동기 대비 12p 감소국내 배터리 3사가 중국을...,니켈 전기차 배터리
139,2024-04-13 14:01:09,배터리 업계 불황에 더 돋보이는 삼성SDI의 깜짝 성장세,삼성SDI 점유율 2023년 932024년 112SK온과 글로벌 순위 뒤바뀌어고품질...,니켈 전기차 배터리
140,2024-04-09 03:05:05,"현대차, 인도서도 배터리 현지화… 가격경쟁력 쑥",인도 배터리 기업과 전략적 동맹 신흥시장서 전기차 리더십 구축도 원료 포괄 방식으로...,니켈 전기차 배터리


In [73]:
df.to_csv('df_뉴스2.csv', index = False)

In [92]:
# 한국어 불용어 리스트 (필요에 따라 추가/제거 가능)
korean_stopwords = ['것으로', '올해','있는','대한','이후','지난해',
    '이', '그', '저', '것', '있', '하', '것들', '들', '에서', '의', '를', '으로', '에', '그리고', '가', '를', '에게',
    '뿐만', '뿐', '이다', '있다', '없다', '에서', '과', '도', '을', '와', '이다', '으로', '이다', '것이다', '더', '등',
    '만', '이다', '또', '한', '가장', '것', '때문에', '로', '에', '이', '들', '잘', '한', '수', '하다'
]


In [93]:
# TF-IDF 분석 및 중요 단어 추출
def tfidf_analysis(df, keywords):
    results = {}
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words=korean_stopwords)
    
    for keyword in keywords:
        keyword_df = df[df['Keyword'] == keyword].reset_index(drop=True)
        if not keyword_df.empty:
            tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_df['Content'])
            feature_names = tfidf_vectorizer.get_feature_names_out()
            tfidf_df = pd.DataFrame(tfidf_matrix.T.toarray(), index=feature_names)
            tfidf_df.columns = keyword_df['title']
            
            # 중요 단어 추출
            keyword_tfidf = tfidf_df.sum(axis=1)
            top_terms = keyword_tfidf.sort_values(ascending=False).head(10)
            
            results[keyword] = top_terms
    
    return results

In [94]:
# TF-IDF 분석
tfidf_results = tfidf_analysis(df, keywords)


In [95]:
# 결과 출력
for keyword, top_terms in tfidf_results.items():
    print(f"\n키워드: {keyword}\n")
    print(top_terms)


키워드: 니켈 원자재

구리     2.594198
원자재    2.296731
니켈     2.152359
선물이    1.992403
오늘     1.894797
가격이    1.836877
배터리    1.798148
중국     1.791390
미국     1.771406
지난     1.749660
dtype: float64

키워드: 니켈 배터리

배터리     4.560852
전기차     2.849216
미국      1.763677
중국      1.729983
배터리를    1.664814
재활용     1.553386
글로벌     1.467886
1분기     1.464215
충전      1.446411
위한      1.270918
dtype: float64

키워드: 니켈 전기차 배터리

배터리     5.657957
전기차     4.888693
중국      2.205145
배터리를    2.164028
미국      2.136786
1분기     2.129336
국내      1.976432
글로벌     1.944299
수요      1.930549
시장      1.893569
dtype: float64


In [96]:
# 모든 DataFrame 및 분석 결과 저장
df.to_csv('all_news_articles.csv', index=False)
for keyword in keywords:
    keyword_df = df[df['Keyword'] == keyword]
    keyword_df.to_csv(f'news_articles_{keyword}.csv', index=False)
    if keyword in tfidf_results:
        tfidf_results[keyword].to_csv(f'top_terms_{keyword}.csv')

# keyword : 니켈 전기차 배터리

In [108]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

start_date = "2024.02.01"
end_date = "2024.04.30"
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
#https://search.naver.com/search.naver?where=news&query={keyword}&pd=3&ds={start_date}&de={end_date}&start={page}
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start="  + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:니켈 전기차 배터리

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):400

크롤링할 종료 페이지:  400 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=1', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=11', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=21', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=31', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=41', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=51', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=61', 'https://search.naver.com/search.naver?where=news&query=니켈 전기차 배터리&pd=

100%|█████████████████████████████████████████| 102/102 [00:21<00:00,  4.85it/s]

검색된 기사 갯수: 총  4000 개

[뉴스 제목]
['"10 Minutes" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑', '투자 더 늘린다는 삼성SDI "전기차 장기 전망 밝아"', "K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷", '중국산 전기차 사고로 폭발, 문 열리지 않아 탑승자 전원 사망', "'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”", '엑셀 밟는 비야디 전기차…"연내 승용차 브랜드 론칭"', "'3천만원대' 기아 EV3 6월 출시…전기차 '캐즘' 돌파 선봉장", '"배터리 가격 경쟁력 갖추자"…이석희 SK온 사장 \'승부수\'', 'K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐', '현대차 전기 SUV ‘아이오닉9’ SK온 고성능 배터리 탑재된다', 'K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진', "현대차·기아, CFO '전기차 동상이몽'...캐스퍼·EV3 전략 온도차", '“폴스타 전기차에 SK온 배터리 탑재·현지 생산…한국 기업과 협력 확대”', "'무임승차'에 뿔난 LG엔솔 '배터리 특허 전쟁' 선언…中 겨냥했나", '車업계, 전기차 신차 잇단 출시…캐즘 정면돌파 의지', 'LG엔솔, 전기차 부진에 결국 투자 규모 줄인다', "'英스포츠카' 로터스 품은 中 지리…'럭셔리 전기차'도 노린다", '"잠깐 쉬었다 멀리 가자"…K-배터리, 수요 살피며 속도조절', '포스코 “이차전지 소재 투자 일부 순연… 전기차 ‘캐즘’ 대응”', "SK온, '코발트 프리 배터리' 에디슨 어워즈 동상 수상", '[인터뷰] 벤츠 최고기술책임자 "전동화전략 여전…전기차 할인경쟁 안해"', "포스코홀딩스, 1분기 영업익 17.3% 감소…배터리 투자 '브레이크'", "'캐즘'에 놀란 장인화號, 배터리 투자 늦추고 부진 사업 구조조정", '[배터리완전정복](33)中 LFP 대적할 3대 키워드…단결정·고전압·미드니켈', '[단독] 현대차 ‘아이오닉9’에 SK온 배터리 탑재된다', "전략특허 58% 뚫렸다…뿔난 L

In [98]:
df1 = news_df.copy() #100행

In [100]:
df2 = news_df.copy() #200행

In [102]:
df3 = news_df.copy() #300행

In [109]:
df4 = news_df.copy() # 400행

In [110]:
# 각 DataFrame의 행 개수를 구하기
row_count_df1 = len(df1)
row_count_df2 = len(df2)
row_count_df3 = len(df3)
row_count_df4 = len(df4)

# 가장 많은 행 수를 가진 DataFrame을 출력
max_rows_df = max([(row_count_df1, 'df1'), (row_count_df2, 'df2'), (row_count_df3, 'df3'), (row_count_df4, 'df4')])[1]
print(f"가장 많은 행 수를 가진 DataFrame은 {max_rows_df}입니다.")

가장 많은 행 수를 가진 DataFrame은 df3입니다.


In [112]:
print(df1.shape, df2.shape, df3.shape, df4.shape)

(81, 4) (88, 4) (95, 4) (81, 4)


In [105]:
df3

,date,title,link,content
0,2024-04-30 14:26:01,"""10 Minutes"" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑",https://n.news.naver.com/mnews/article/018/000...,[\n주행 가능한 폴스타 5 프로토타입에 시연10분 만에 중형 전기차 주행거리 32...
1,2024-04-30 13:09:01,"투자 더 늘린다는 삼성SDI ""전기차 장기 전망 밝아""",https://n.news.naver.com/mnews/article/092/000...,[\n美 공급확대로 '캐즘' 돌파 위기헷징…IRA 수혜 내년 본격 확대 전망삼성SD...
2,2024-04-30 13:14:19,"K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷",https://n.news.naver.com/mnews/article/001/001...,[\n고객사 보수적 재고 운영에 메탈가 하락…수익성에 악영향하반기 신차 출시·美 A...
3,2024-04-30 10:59:01,"중국산 전기차 사고로 폭발, 문 열리지 않아 탑승자 전원 사망",https://n.news.naver.com/mnews/article/022/000...,"[\n사고 후 열폭주 현상, 문 안 열려 참극 \n\n\n\n 추돌사고 후 화염..."
4,2024-04-30 16:43:01,'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”,https://n.news.naver.com/mnews/article/011/000...,"[\n■1분기 부진 K배터리, 회복 시점은영업익 29% 감소하며 '상대적 선방'LG..."
...,...,...,...,...
90,2024-02-20 16:55:01,테슬라 195만원 Vs 현대차 690만원…계속되는 전기차 보조금 차별 논란,https://n.news.naver.com/mnews/article/018/000...,"[\n환경부, 전기차 차종별 보조금 확정 고시외국산 車·배터리 불리…테슬라·KGM ..."
91,2024-02-21 08:44:52,"LFP 배터리 때린 환경부, 중국 공세 막아낼까 [신인규의 이슈레이더]",https://n.news.naver.com/mnews/article/215/000...,"[\n\n\n\n\n●LFP 배터리 때린 환경부, 중국 공세 막아낼까환경부의 전기차..."
92,2024-02-22 12:02:11,"""韓, '中대체' 배터리 공급망 허브 잠재력…국가전략 필요""",https://n.news.naver.com/mnews/article/018/000...,"[\n대한상의, '한국 배터리 공급망 허브 구축가능성' 연구""양극재 최대 수출국이나..."
93,2024-02-20 15:20:49,올해 전기차 보조금 보니…국산차 ‘맑음’·수입차 ‘흐림’,https://n.news.naver.com/mnews/article/056/001...,[\n\n\n\n\n■현대차 아이오닉 모델 8개종 '690만 원'.. 보조금 액수 ...


In [106]:
# 1. pDate 열을 datetime 형식으로 변환
df3['date'] = pd.to_datetime(df3['date'])
# 2. pDate 열을 기준으로 데이터프레임을 날짜순으로 정렬
df3 = df3.sort_values(by='date')

In [107]:
df3.to_csv('니켈_뉴스크롤링_최종.csv', index = False)